In [1]:
import torch

In [5]:
import sys
sys.path

['D:\\Anaconda\\envs\\learn_pytorch_39\\python39.zip',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\DLLs',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\lib',
 'D:\\Anaconda\\envs\\learn_pytorch_39',
 '',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\lib\\site-packages',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\lib\\site-packages\\win32',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\lib\\site-packages\\win32\\lib',
 'D:\\Anaconda\\envs\\learn_pytorch_39\\lib\\site-packages\\Pythonwin']

In [6]:
torch.__version__

'2.8.0+cu126'

In [14]:
torch.cuda.is_available()

True

In [ ]:
print("hello world")

### pytorch 里面没有类型支持string
所以对于 NLP 会使用Embedding 的一些向量库 如 word2vec

In [9]:
a = torch.tensor(1)
b = torch.tensor(1.)
print(b.type())
print(a.type())
type(a)

torch.FloatTensor
torch.LongTensor


torch.Tensor

In [16]:
a = a.cuda()
isinstance(a,torch.cuda.LongTensor)

True

dimension = 0 的tensor 一般用来表示loss
dimension = 1 的就是向量或者标量，在python里面就是一维数组 一般用来表示神经网络中的bias
创建方法：
- 从列表创建  torch.tensor([1,2,3,4])
- 从类型创建 给出维度 torch.FloatTensor(2)
- 从numpy 引入 torch.from_numpy(data)
- 随机初始化
  - torch.rand(size) 创建[0,1) 的随机取值
  - torch.rand_like(tensor) 接受的参数是一个tensor变量 形状和tensor的一样
  - torch.randn(size) 创建的是 均值为0 方差为1 的正态分布 常用于各种权重的初始化
  - torch.randint(min,max,shape) [min,max) 整数
- 注意 torch.Tensor() 跟 torch.FloatTensor() 作用是一样的，参数都是shape torch.tensor(list) 参数是list
- torch.empty(shape) 创建一块内存空间，但是不赋值
- torch.arange(start,end,step)  与python 中的range 一样
- torch.linspace(start,end,num)  [start,end] num 个数，这里不一样，首先是 end闭区间，其次是 不是步长，而是个数

In [ ]:
a.shape,a.dim()

(torch.Size([]), 0)

In [ ]:
a = torch.FloatTensor(2,3) # 两行三列
a

tensor([[6.8824e-21, 1.9212e-42, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [22]:
import numpy as np
b = np.ones((2,3))
b = torch.from_numpy(b)
b

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

In [26]:
c = torch.linspace(1,10,7)
c

tensor([ 1.0000,  2.5000,  4.0000,  5.5000,  7.0000,  8.5000, 10.0000])

### 切片和索引
- torch.index_select(dim,[])
- 多个 :,:,: 可以省略称... 
- torch.masked_select(tensor,mask)  感觉很像是bool索引

In [28]:
a = torch.randn(4,3,28,28)
a.shape

torch.Size([4, 3, 28, 28])

In [30]:
a[:,2,...].shape

torch.Size([4, 28, 28])

In [31]:
a[:,1].shape

torch.Size([4, 28, 28])

In [33]:
x = torch.randn(3,4)
x

tensor([[ 0.9906,  1.5477, -2.0208,  0.9014],
        [ 0.2829,  0.1293, -0.6898, -0.5056],
        [ 0.9746,  0.8371, -0.8640,  0.3805]])

In [35]:
mask = x.ge(0.5) # 大于
mask

tensor([[ True,  True, False,  True],
        [False, False, False, False],
        [ True,  True, False, False]])

In [36]:
torch.masked_select(x,mask)

tensor([0.9906, 1.5477, 0.9014, 0.9746, 0.8371])

### 维度变换
- View reshape 功能完全相同，直接用reshape 就行
- unsqueeze(dim) 在dim 增加一个维度，其他维度向后移动
- squeeze() 默认是将为1的维度都去掉 squeeze(dim) 如果shape 不是1 则不会变
- expand ： broadcasting 不会立刻复制，只在用到的时候才进行复制
- repeat : 直接进行复制（ 都是只有dim = 1 时才会进行复制）（需要不变可以写-1）
    - 注意 expand(shape) 参数是扩展后的shape，而repeat() 中的参数是每个维度重复的次数
- .t() 转置，只能适用于二维的矩阵
- .transpose(dim1,dim2) 用来交换连个维度 
    - 注意：使用之后，数据的内存就会变得不连续，就不能使用view(但是可以在前面加上contiguous())  但可以使用reshape
- 。permute()  参数就是你想要的之前维度的排列方式

In [43]:
a = torch.randn(4,3,28,28)
a.shape,a.transpose(1,3).contiguous().view(4,3*28*28)

(torch.Size([4, 3, 28, 28]),
 tensor([[ 1.5289,  0.6486,  0.0149,  ..., -0.7132, -1.4077,  0.0785],
         [ 1.0363, -0.7139,  0.6095,  ..., -1.2027, -0.6341, -0.0110],
         [ 1.8179, -1.1500, -0.5700,  ...,  0.2262, -0.6996, -0.5316],
         [ 1.5854,  0.5165,  0.3571,  ...,  0.4487, -1.2539,  0.7470]]))

### Merge or Split
- cat 合并 torch.cat([tensor1,tensor2],dim = ) 要求只有dim 那个维度数字可以不一样，其他必须一样
- stack([tensor1,tensor2],dim = ) 要求维度完全相同，然后在dim上添加一个维度

In [48]:
# [calss，stuednt，course] 学生成绩单
a = torch.randn(4,32,8)
b = torch.randn(5,32,8)
c = torch.randn(5,24,8)
torch.cat([a,b],dim = 0).shape ,torch.cat([c,b],dim = 1).shape


(torch.Size([9, 32, 8]), torch.Size([5, 56, 8]))

In [51]:
a1 = torch.randn(4,32,8)
a2 = torch.randn(4,32,8)
torch.stack([a1,a2],dim = 1).shape

torch.Size([4, 2, 32, 8])

In [53]:
c = torch.rand(3,32,8)
a1,a2 = c.split([1,2],dim = 0)
a1.shape,a2.shape

(torch.Size([1, 32, 8]), torch.Size([2, 32, 8]))

### tensor 的基本运算
- torch.mm() only for 2d
- torch.matmul() 矩阵相乘 @ 一样 对于高纬度的矩阵 matmul 还是只取最后两个维度进行运算
- torch.exp()
- torch.log() log2() log10()
- .floor() 向下取整 .ceil() 向上取整 .round() 四舍五入 .trunc()截取数字的整数部分 .frac() 截取数字的小数部分
- .clamp(min) .clamp(min,max) 设置一个最小值和最大值 小于min的 都变成min
    — 常用在梯度爆炸的时候，给梯度限制一个最大值

In [58]:
a = torch.rand(4,3)
b = torch.rand(3,5)
torch.matmul(a,b)

tensor([[0.5927, 0.7599, 1.1065, 1.0511, 0.4303],
        [0.6924, 0.8996, 1.2622, 1.2823, 0.7034],
        [0.1371, 0.5106, 0.7409, 0.6820, 0.6451],
        [0.7937, 0.8772, 1.2085, 1.2807, 0.6140]])

### tensor 的统计属性 
- 范数 norm 范数是向量或者矩阵大小的度量，将向量映射到非负实数的函数，它可以衡量向量的大小或长度
    - torch.nn.functional.normalize 用于对输入张量进行归一化处理，也就是将输入张量的每个元素除以其范数，使得处理后的张量的范数为 1。
    - 第一范数是所有元素绝对值求和
    - norm = 2 是所有元素平方和再开根号
- 也可以在某个维度上求范式 norm(p = ,dim = )
- mean() 平均值 sum() min() prod() 累乘
- argmax() argmin() 返回的是索引，是最大元素的索引或者最小元素的索引 
    - 如果是高维的，且不指定维度，则会打平，
    - 指定哪个维度，哪个维度就会被压缩，变成一个值
    - 一般可以配合max 一起使用 max(dim = ,keepdim = Trye or False)
- topk(num,dim,largest = ) largets = False 表示最小的几个
- torch.equal() 只返回一个True or False torch.eq() =  ==



In [64]:
a = torch.arange(10)
b = torch.arange(10)
torch.equal(a,b)

True

### 激活函数
- sigmoid函数，也叫logistic 函数，所以这也是logistic regression 的由来
- sigmoid函数有一个很特别的性质  
$ sigmoid = \dfrac{1}{1 + e^{-x}} $    
$ y' = y(1-y)$   
所以求导很快  
- 有个问题，就是如果x特别大或者x特别小说，会过于平缓，这就是梯度离散或者梯度消失
- 使用方法 import nn.functional as F   F.sigmoid() torch.sigmoid()

- Tanh 激活函数
\begin{align}
f(x) = tanh(x) &= \dfrac{e^x - e^{-x}}{e^x + e^{-x}} \\
&= 2sigmoid(2x)-1
\end{align}

- softmax() 激活函数
$ y = \dfrac{exp(o_i)}{\sum{}_{k}exp(o_k)} $   
softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持 可导的性质  

softmax 激活函数会放大差距，而且各个概率的总和为1 特别适合用来分类问题


In [73]:
import torch.nn.functional as F
F.sigmoid(torch.tensor(5))

tensor(0.9933)

### 损失函数Loss
- 均方差 MSE Mean Squared Error
- Cross Entropy loss 交叉熵损失函数
    - 交叉熵常用来衡量两个概率的区别 如现在有两个概率分布p 和q  
    $ H(p,q) = \sum_{i}-p_{i}log(q_i)$  

交叉熵 可以想象为 主观概率为Q 的观察者在看到根据概率P生成的数据时的预期惊讶程度 （主观概率Q 就是Label P就是预测值

### 自动求导 torch.autograd.grad(output,input) 与 output.backward()
- autograd 不会累计梯度，会将梯度的计算结果直接返回 .backward()不会返回梯度，而是累计在.grad里面
- torch.autograd.grad(loss,[w1,w2,w3])  loss.backward() w1.grad w2.grad

### Visdom 可视化

 